---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 2 - Network Connectivity

In this assignment you will go through the process of importing and analyzing an internal email communication network between employees of a mid-sized manufacturing company. 
Each node represents an employee and each directed edge between two nodes represents an individual email. The left node represents the sender and the right node represents the recipient.

In [46]:
import networkx as nx

# This line must be commented out when submitting to the autograder
# !head email_network.txt

### Question 1

Using networkx, load up the directed multigraph from `email_network.txt`. Make sure the node names are strings.

*This function should return a directed multigraph networkx graph.*

In [47]:
def answer_one():
    
    # Read in graph
    G = nx.read_edgelist('email_network.txt', nodetype=str, delimiter='\t', 
                         create_using=nx.MultiDiGraph(), data=[('time', int)])
    
    return G

### Question 2

How many employees and emails are represented in the graph from Question 1?

*This function should return a tuple (#employees, #emails).*

In [48]:
def answer_two():
        
    # Employees are nodes and emails are edges
    G = answer_one()
    
    return (len(G.nodes()), len(G.edges()))

### Question 3

* Part 1. Assume that information in this company can only be exchanged through email.

    When an employee sends an email to another employee, a communication channel has been created, allowing the sender to provide information to the receiver, but not vice versa. 

    Based on the emails sent in the data, is it possible for information to go from every employee to every other employee?


* Part 2. Now assume that a communication channel established by an email allows information to be exchanged both ways. 

    Based on the emails sent in the data, is it possible for information to go from every employee to every other employee?


*This function should return a tuple of bools (part1, part2).*

In [49]:
def answer_three():
        
    # Part 1: Is the graph strongly connected?
    # Part 2: Is the graph weakly connected?
    
    G = answer_one()
    
    return (nx.is_strongly_connected(G), nx.is_weakly_connected(G))

### Question 4

How many nodes are in the largest (in terms of nodes) weakly connected component?

*This function should return an int.*

In [50]:
def answer_four():
    
    # Sort the weakly_connected_components of G and obtain the length 
    G = answer_one()
    
    return len(sorted(nx.weakly_connected_components(G))[0])

### Question 5

How many nodes are in the largest (in terms of nodes) strongly connected component?

*This function should return an int*

In [51]:
def answer_five():
    
    G = answer_one()
    
    return len(max(sorted(nx.strongly_connected_components(G)), key=len))

### Question 6

Using the NetworkX function strongly_connected_component_subgraphs, find the subgraph of nodes in a largest strongly connected component. 
Call this graph G_sc.

*This function should return a networkx MultiDiGraph named G_sc.*

In [52]:
def answer_six():
        
    G = answer_one()
    
    G_sc = max(nx.strongly_connected_component_subgraphs(G), key=len)
    
    return G_sc

### Question 7

What is the average distance between nodes in G_sc?

*This function should return a float.*

In [53]:
def answer_seven():
        
    G_sc = answer_six()
    
    ans = nx.average_shortest_path_length(G_sc)
    
    return ans

### Question 8

What is the largest possible distance between two employees in G_sc?

*This function should return an int.*

In [54]:
def answer_eight():
        
    G_sc = answer_six()
    
    ans = nx.diameter(G_sc)
    
    return ans

### Question 9

What is the set of nodes in G_sc with eccentricity equal to the diameter?

*This function should return a set of the node(s).*

In [55]:
def answer_nine():
       
    G_sc = answer_six()
    
    ans = nx.periphery(G_sc)
    
    ans = set(ans)
    
    return ans

### Question 10

What is the set of node(s) in G_sc with eccentricity equal to the radius?

*This function should return a set of the node(s).*

In [56]:
def answer_ten():
        
    G_sc = answer_six()
    
    ans = nx.center(G_sc)
    
    ans = set(ans)
    
    return ans

### Question 11

Which node in G_sc is connected to the most other nodes by a shortest path of length equal to the diameter of G_sc?

How many nodes are connected to this node?


*This function should return a tuple (name of node, number of satisfied connected nodes).*

In [57]:
def answer_eleven():
       
    G_sc = answer_six()
    diameter = answer_eight()
    
    # Iterated over all nodes from advice in forums as for directed graph, if nodes 'A' and 'B' have path from 
    # 'A' to 'B' equal to diameter of graph, then 'A' will be on the periphery, but 'B' may not if the path from
    # 'B' to 'A' is less than the diameter. However, my previous method would have worked as it went through all
    # paths to all nodes from just the nodes on the periphery, so it would've been quicker
    
    # Also the question appears to ask for the number of nodes connected to the node that is connected to the most
    # other nodes by a shortest path length equal to graph diameter, however autograder accepts answer instead as
    # the number of paths that have shortest path length equal to graph diameter

    ans_dict = {}
    path_dict = nx.shortest_path(G_sc)
    for node in path_dict:
        paths = path_dict[node]
        path_list = []
        for path in paths.values():
            if len(path) == diameter + 1:
                path_list.append(path)
        if len(path_list) > 0:
            ans_dict[node] = path_list
            
    max_node = max(ans_dict)
    num_paths = len(ans_dict[max_node])
    
    return (max_node, num_paths)

### Question 12

Suppose you want to prevent communication from flowing to the node that you found in the previous question from any node in the center of G_sc, what is the smallest number of nodes you would need to remove from the graph (you're not allowed to remove the node from the previous question or the center nodes)? 

*This function should return an integer.*

In [58]:
def answer_twelve():
        
    G_sc = answer_six()
    center_nodes = answer_ten()
    end_node = answer_eleven()[0]
    
    # Set this high so a smaller value will be found
    nodes_to_remove = 1000
    
    # Iterate over center nodes, find the number of nodes that would need removed to remove path between start
    # and end node, and only keep the start node and num of least amount of nodes needing removed
    for start_node in center_nodes:
        current_min_nodes = nx.node_connectivity(G_sc, start_node, end_node)
        if current_min_nodes < nodes_to_remove:
            nodes_to_remove = current_min_nodes
            min_node_name = start_node
            
    # Question is apparantly flawed as center node (only 1) has an edge directly to end node
    # Therefore the below is necessary according to forums
    
    return len(nx.minimum_node_cut(G_sc, min_node_name, end_node))

### Question 13

Construct an undirected graph G_un using G_sc (you can ignore the attributes).

*This function should return a networkx Graph.*

In [59]:
def answer_thirteen():
        
    G_sc = answer_six()
    G_un = G_sc.to_undirected()
    G_un = nx.Graph(G_un)
    
    return G_un

### Question 14

What is the transitivity and average clustering coefficient of graph G_un?

*This function should return a tuple (transitivity, avg clustering).*

In [60]:
def answer_fourteen():
        
    G_un = answer_thirteen()
    
    transitivity = nx.transitivity(G_un)
    avg_clustering = nx.average_clustering(G_un)
    
    return (transitivity, avg_clustering)